In [22]:
import pandas as pd
url = 'https://raw.githubusercontent.com/gchandra10/filestorage/refs/heads/main/stock_market.csv'
df = pd.read_csv(url)
df.head()


,Trade Date,Ticker,Open Price,Close Price,Volume,Sector,Validated,Currency,Exchange,Notes
0,09/01/2025,AAPL,192.47,191.90,21646536,Technology,na,-,NASDAQ,gap up
1,09/01/2025,MSFT,456.51,459.76,NaN,Technology,-,na,na,NaN
2,09/01/2025,NaN,151.27,150.45,20405836,Consumer Discretionary,na,usd,NASDAQ,-
3,09/01/2025,-,139.61,141.17,12795579,-,Yes,NaN,NYSE,gap down
4,09/01/2025,-,1190.21,1198.56,16389274,Semiconductors,na,USD,NYSE,NaN


In [23]:
print(df.shape)
df.info()
df.isnull().sum()


(200, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Trade Date   200 non-null    object
 1   Ticker       185 non-null    object
 2   Open Price   188 non-null    object
 3   Close Price  191 non-null    object
 4   Volume       188 non-null    object
 5   Sector       191 non-null    object
 6   Validated    182 non-null    object
 7   Currency     127 non-null    object
 8   Exchange     189 non-null    object
 9   Notes        146 non-null    object
dtypes: object(10)
memory usage: 15.8+ KB


Trade Date      0
Ticker         15
Open Price     12
Close Price     9
Volume         12
Sector          9
Validated      18
Currency       73
Exchange       11
Notes          54
dtype: int64

In [24]:
import re
df.columns = [re.sub(r'\W+', '_', c.strip()).lower() for c in df.columns]


In [25]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


C:\Users\Gowtham reddy\AppData\Local\Temp\ipykernel_27788\1718083031.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [26]:
BAD_VALS = ["", "NA", "N/A", "null", "-", "na"]
df = df.replace(BAD_VALS, pd.NA)

In [28]:
df['ticker'] = df['ticker'].str.upper()
df['sector'] = df['sector'].str.strip().str.title()
df['trade_date'] = pd.to_datetime(df['trade_date'], errors='coerce').dt.strftime('%Y-%m-%d')
df['open_price'] = pd.to_numeric(df['open_price'], errors='coerce')
df['close_price'] = pd.to_numeric(df['close_price'], errors='coerce')
df['volume'] = pd.to_numeric(df['volume'], errors='coerce')


In [29]:
df = df.drop_duplicates()

In [30]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   trade_date   200 non-null    object 
 1   ticker       158 non-null    object 
 2   open_price   163 non-null    float64
 3   close_price  164 non-null    float64
 4   volume       156 non-null    float64
 5   sector       173 non-null    object 
 6   validated    114 non-null    object 
 7   currency     55 non-null     object 
 8   exchange     164 non-null    object 
 9   notes        127 non-null    object 
dtypes: float64(3), object(7)
memory usage: 15.8+ KB


,trade_date,ticker,open_price,close_price,volume,sector,validated,currency,exchange,notes
0,2025-09-01,AAPL,192.47,191.90,21646536.0,Technology,<NA>,<NA>,NASDAQ,gap up
1,2025-09-01,MSFT,456.51,459.76,NaN,Technology,<NA>,<NA>,<NA>,NaN
2,2025-09-01,NaN,151.27,150.45,20405836.0,Consumer Discretionary,<NA>,usd,NASDAQ,<NA>
3,2025-09-01,<NA>,139.61,141.17,12795579.0,<NA>,Yes,NaN,NYSE,gap down
4,2025-09-01,<NA>,1190.21,1198.56,16389274.0,Semiconductors,<NA>,USD,NYSE,NaN


In [42]:
df.to_parquet('cleaned.parquet')

In [43]:
agg1 = df.groupby(['trade_date', 'ticker'])['close_price'].mean().reset_index()
agg1.to_parquet('agg1.parquet')

In [44]:
agg2 = df.groupby('sector')['volume'].mean().reset_index()
agg2.to_parquet('agg2.parquet')

In [45]:
df_sorted = df.sort_values(['ticker', 'trade_date'])
df_sorted['prev_close'] = df_sorted.groupby('ticker')['close_price'].shift(1)
df_sorted['daily_return'] = (df_sorted['close_price'] - df_sorted['prev_close']) / df_sorted['prev_close']
agg3 = df_sorted.groupby(['trade_date', 'ticker'])['daily_return'].mean().reset_index()
agg3.to_parquet('agg3.parquet')
